In [1]:
from core.meta import meta_interface
%matplotlib inline

C:\Users\Kobel_K\.conda\envs\pipeline_b06\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
d = meta_interface(project_config_filepath="Z:Konstantin/project_config_server.yaml", project_name="analysis_01")

In [3]:
d.add_recording_config("Z:Konstantin/Calibration/220921/recording_config_210922.yaml")
d.add_recording_config("Z:Konstantin/Calibration/220922/recording_config_220922.yaml")
d.add_recording_config("Z:Konstantin/Calibration/220923/recording_config_230922.yaml")
#d.add_recording_config("Z:Konstantin/Calibration/220927/recording_config_270922.yaml")
#some recordings that are unnecessary
#d.add_recording_config("Z:Konstantin/Calibration/221219/recording_config_191222.yaml")
#d.add_recording_config("Z:Konstantin/Calibration/221220/recording_config_201222.yaml")
#d.add_recording_config("Z:Konstantin/Calibration/221221/recording_config_211222.yaml")
#d.add_recording_config("Z:Konstantin/Calibration/221222/recording_config_221222.yaml")

In [4]:
d.initialize_meta_config()

In [5]:
d.create_recordings()

In [ ]:
d.load_meta_from_yaml(filepath="Z:Konstantin/analysis_01.yaml")

In [6]:
d.synchronize_recordings(verbose = True, test_mode = False)  # test_mode: bool, doesnt overwrite synchronized and analysed videos
# exclude recordingvideos based on synchronization

100%|███████████████████████████████████████████████████████████████████████████▉| 47400/47438 [09:57<00:00, 79.39it/s]


KeyboardInterrupt: 

In [ ]:
d.load_meta_from_yaml(filepath="test_data/My_project.yaml")

In [ ]:
d.create_calibrations()
# update fps, video_filepath, target_fps

In [ ]:
d.synchronize_calibrations(test_mode = True)  

# test_mode: bool, doesnt overwrite synchronized videos
# to speed up testing, remove some cams from valid_cam_IDs in project_config.yaml

# exclude calibrationvideos based on synchronization, change markerdetection_filepaths etc.

In [ ]:
d.calibrate()
# exclude calibrations based on calibration log and repeat calibration (not yet implemented) 

In [ ]:
d.triangulate_recordings()

In [ ]:
d.meta

In [ ]:
d.export_meta_to_yaml(filepath="test_data/My_project.yaml")

In [ ]:
d.load_meta_from_yaml(filepath="test_data/My_project.yaml")

In [5]:
from pathlib import Path
from core.triangulation_calibration_module import (
    Calibration,
    Triangulation_Positions,
    Triangulation_Recordings,
)

In [9]:
rec_config = Path("/Users/kobel/Downloads/220922/recording_config_220922.yaml")
calibration_object = Calibration(calibration_directory=rec_config.parent, 
                                 output_directory=rec_config.parent, 
                                 recording_config_filepath=rec_config, 
                                 project_config_filepath = "test_data/project_config.yaml")

In [10]:
calibration_object.run_synchronization(test_mode = True)

100%|█████████████████████████████████████████| 100/100 [01:08<00:00,  1.45it/s]


In [11]:
calibration_object.run_calibration(verbose=True)

100%|██████████████████████████| 18447/18447 [00:30<00:00, 613.85it/s]


In [2]:
rec_config = Path("/Users/kobel/Downloads/220927/recording_config_270922.yaml")
calibration_object = Calibration(calibration_directory=rec_config.parent, 
                                 output_directory=rec_config.parent, 
                                 recording_config_filepath=rec_config, 
                                 project_config_filepath = "test_data/project_config.yaml")
calibration_object.run_synchronization(test_mode = True)

2023-01-19 08:56:57.099082: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-01-19 08:56:57.101259: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-19 08:56:57.153091: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:196] None of the MLIR optimization passes are enabled (registered 0 passes)
 85%|████████████████████████████████████▌      | 40/47 [00:31<00:05,  1.29it/s]


possibly bad synchronization!
/Users/kobel/Downloads/220927/220927_Side3_synchronized.mp4
3243 boards detected
/Users/kobel/Downloads/220927/220927_Side2_synchronized.mp4
6069 boards detected
/Users/kobel/Downloads/220927/220927_Side1_synchronized.mp4
3526 boards detected
/Users/kobel/Downloads/220927/220927_Bottom_synchronized.mp4
4750 boards detected
/Users/kobel/Downloads/220927/220927_Ground2_synchronized.mp4
5564 boards detected
/Users/kobel/Downloads/220927/220927_Ground1_synchronized.mp4
4863 boards detected
/Users/kobel/Downloads/220927/220927_Top_synchronized.mp4
0 boards detected
defaultdict(<class 'int'>,
            {('Bottom', 'Ground1'): 2112,
             ('Bottom', 'Ground2'): 1680,
             ('Bottom', 'Side1'): 149,
             ('Bottom', 'Side2'): 389,
             ('Bottom', 'Side3'): 695,
             ('Ground1', 'Bottom'): 2112,
             ('Ground1', 'Ground2'): 2015,
             ('Ground1', 'Side1'): 217,
             ('Ground1', 'Side2'): 428,
          

In [3]:
calibration_object.run_calibration(verbose=True)

100%|███████████████████████████████| 89/89 [00:00<00:00, 1310.31it/s]


AssertionError: Invalid points shape, first dim should be equal to number of cameras (7), but shape is (6, 63567, 2)

In [ ]:
21.09. Side3 to be excluded! do calibration!
22.09. 
23.09. Top to be excluded!
27.09.
19.12. error 0.95
20.12. Side3 not well synchronized, exclude! redo calibration!
21.12. Side2 and Side3 not well synchronized, exclude! redo calibration! Top has 15000 frames less (why?, how?)
22.12. error 2.22 Side3 not well synchronized, exclude! redo calibration!
# calibration error wenn eine Kamera 0 boards detektiert hat!

In [ ]:
a = Calibration(
    calibration_directory="test_data/Server_structure/Calibrations/220826/",
    project_config_filepath="test_data/project_config.yaml",
    recording_config_filepath="test_data/Server_structure/Calibrations/220826/recording_config.yaml",
    output_directory="test_data/Server_structure/Calibrations/220826/",
)

In [ ]:
a.run_synchronization()

In [ ]:
a.run_calibration(verbose=True)

In [ ]:
b = Triangulation_Recordings(
    recording_directory="test_data/Server_structure/VGlut2-flp/September2022/196_F7-27/220826_OTT/",
    calibration_directory="test_data/Server_structure/Calibrations/220826/",
    recording_config_filepath="test_data/Server_structure/Calibrations/220826/recording_config.yaml",
    project_config_filepath="test_data/project_config.yaml",
    output_directory="test_data/Server_structure/VGlut2-flp/September2022/196_F7-27/220826_OTT/",
)

In [ ]:
b.run_synchronization()

In [ ]:
b.run_triangulation(
    calibration_toml_filepath="test_data/Server_structure/Calibrations/220826/calibration_220826.toml"
)

In [ ]:
c = Triangulation_Positions(
    positions_directory="test_data/Server_structure/Calibrations/220826/",
    calibration_directory="test_data/Server_structure/Calibrations/220826/",
    recording_config_filepath="test_data/Server_structure/Calibrations/220826/recording_config.yaml",
    project_config_filepath="test_data/project_config.yaml",
    output_directory="test_data/Server_structure/Calibrations/220826/",
)

In [ ]:
c.get_marker_predictions()

In [ ]:
c.run_triangulation(
    calibration_toml_filepath="test_data/Server_structure/Calibrations/220826/calibration_220826.toml"
)

In [ ]:
# special dependencies

In [ ]:
"""
pip install aniposelib
pip install ffmpeg
pip install ffmpeg-python
pip install imageio-ffmpeg
jupyter lab
Tkinter
"""